# Lesson 5 - Test with Other Datasets

What have we done so far:
1. Use lidar to measure clutter heights (lesson 2).
2. Build a clutter model that considers _3D Clutter Distance_ (lesson 3).
3. Fit the clutter model to real measurement data from the Martin Acres neighborhood (lesson 3).
4. Compare the clutter model to the ITU-R P.2108 clutter model (lesson 4).

In this lesson we will examine how the model predicts clutter loss in a new area, Salt Lake City. To do this we will use three measurement datasets performed on 7/10/2023. These datasets come from NTIA's dataset repository __add a live link__. We will walk through loading the new datasets and then conclude by posing a challenge to all attendees. 

### Measurements from a new clutter environment, Salt Lake City

<img src="./images/ma_slc.png" alt="clutter compare" width="600"/>

Salt Lake City (SLC) provides a new clutter type to examine the performance of our model. There are three measurement datasets at 3.475 GHz taken in SLC. These datasets each have a different TX location (Browning, City Creek, or Ensign) and the same RX locations (within downtown SLC). The files are located in the `course-materials/data/` directory.

- **`course-materials/data/SaltLakeCity_Urban_Ensign_3475_20230710`**
- **`course-materials/data/SaltLakeCity_Urban_CityCreek_3475_20230710`**
- **`course-materials/data/SaltLakeCity_Urban_Browning_3475_20230710`**

### Ensign, Excess Loss

<img src="./images/[map] Ensign Excess Loss, Entire Drive Route 3475 MHz.png" alt="Ensign Map" width="700"/>

### City Creek, Excess Loss

<img src="./images/[map] City Creek Excess Loss, Entire Drive Route 3475 MHz.png" alt="City Creek Map" width="700"/>

### Browning, Excess Loss

<img src="./images/[map] Browning Excess Loss, Entire Drive Route 3475 MHz.png" alt="Browning Map" width="700"/>



# **The Challenge**  

The model we created considers the height of the clutter and the receive angle. In practice, we've shown it is effective at predicting clutter loss in the Martin Acres neighborhood with two different transmit elevations. Our hope is that it will also be effective in areas with different clutter heights and transmit elevations.

In this challenge we ask you, "In what ways does the model fail when tested against the Salt Lake City datasets? What can be done to improve it?"

### Get started by importing the necessary python libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rasterio

## Find clutter heights using SLC Lidar data

Just like lesson 2, let's find the distribution of clutter heights within downtown SLC. The SLC terrain and surface data can be found in the `course-materials/data/` directory.

In [ ]:
## bring in LiDAR data for SLC Downtown
dataset_terrain = rasterio.open("./data/SLC.dtm.tif")
dataset_surface = rasterio.open("./data/SLC.dsm.tif")
band1_terrain = dataset_terrain.read(1)
band1_surface = dataset_surface.read(1)
## set map and LiDAR bounds
left, bottom, right, top = dataset_terrain.bounds
## EPSG 26912
left_deg, bottom_deg, right_deg, top_deg = (-111.9061785, 40.7515306, -111.8817738, 40.7764524)
## for converting lat and long degrees into meters (same as lesson 2)
meters_per_lat = 111319.49 
meters_per_long = 85263.24 ## only valid at latitudes of 40 degrees

### Pick a Latitude and Longitude point for the center of Downtown SLC 

In [ ]:
## takes a lat, long coordinate pair
##  and returns a meters based coordinate pair
def convert_gps_to_meters(lat, long):
    lat_dif = top_deg - lat
    long_dif = long - left_deg
    vert_dif_m = lat_dif * meters_per_lat
    horz_dif_m = long_dif * meters_per_long
    return (vert_dif_m, horz_dif_m)

## get_elev function finds the elevation of a Latitude and Longitude location 
##  takes lat and long of the location, lidar dataset (DSM or DTM), and the lidar band that holds elevation
def get_elev(lat, long, lidar_model, lidar_elev_band):
    vert_dif_m, horz_dif_m = convert_gps_to_meters(lat, long)
    row, col = lidar_model.index(left + horz_dif_m, top - vert_dif_m) ## get the row and colomn where the elevation is stored
    elev = lidar_elev_band[row][col] ## retrieve the elevation
    return elev

## set SLC Downtown center location
slc_center = (40.7614,-111.8925)
slc_center_meters = convert_gps_to_meters(slc_center[0], slc_center[1])
print("SLC center point is at x = {:.2f}, y = {:.2f} (in meters). The origin is the top left corner of lidar data.".format(slc_center_meters[1], slc_center_meters[0]))

### Find clutter heights within 500 meters of the center point in Downtown SLC

In [ ]:
xmid = slc_center_meters[1]
ymid = slc_center_meters[0]
radius = 500
## generate 10000 random x,y locations near the SLC center
x_random = np.random.randint(xmid-radius, xmid+radius, 10000)
y_random = np.random.randint(ymid-radius, ymid+radius, 10000)

## create lists to hold the locations and clutter heights that are within 500 meters of the center point
within_500m_x = []
within_500m_y = []
clutter_height_ls = []

## loop through all of the random points
for i in range(len(x_random)):
    ## find distance from center to a random point
    distance = np.sqrt(np.square(slc_center_meters[1]-x_random[i]) + np.square(slc_center_meters[0]-y_random[i]))
    
    ## if distance is within 500 meters, find the clutter height at that location
    if distance < radius:
        ground_elev = band1_terrain[y_random[i]][x_random[i]]
        clutter_elev = band1_surface[y_random[i]][x_random[i]]
        clutter_height = clutter_elev - ground_elev
        ## clutter must be higher than 2 meters to be considered "clutter"
        if clutter_height > 2:
            clutter_height_ls.append(clutter_height)
            within_500m_x.append(x_random[i])
            within_500m_y.append(y_random[i])

### Plot the distribution of clutter heights in Downtown SLC

In [ ]:
plt.rcParams["figure.figsize"] = (10,5)

# the histogram of the data
n, bins, patches = plt.hist(clutter_height_ls, np.linspace(2,40,39), density=True, facecolor='g', alpha=.85)


plt.xlabel('Clutter Height (meters)')
plt.ylabel('Probability')
plt.title('Histogram of Clutter Heights of Salt Lake City Downtown')
plt.grid(True)
plt.show()

In [ ]:
print("Downtown SLC Clutter Heights:\n Mean = {:.1f} meters\n Median = {:.1f} meters\n Standard Deviation = {:.1f} meters".format(np.mean(clutter_height_ls),
                                                                                                                         np.median(clutter_height_ls),
                                                                                                                         np.std(clutter_height_ls)))

slc_clutter_height_mean = np.mean(clutter_height_ls)
slc_clutter_height_std = np.std(clutter_height_ls)

# Dataset #1: Ensign

It's time to look at the measurement data gathered from the Ensign TX location. Start by loading the data from CSV into a DataFrame.

In [ ]:
## load the measurement data
slc_ensign_name = "./data/SaltLakeCity_Urban_Ensign_3475_20230710.csv"
slc_ensign_df_full = pd.read_csv(slc_ensign_name)
slc_ensign_df = slc_ensign_df_full.where(slc_ensign_df_full["ID"] < 54638) ## ignore measurements after ID 54638 (transmitter was turned off)

## set static variables like TX location, frequency, and antenna heights
ensign_TxLat = 40.7910862900222
ensign_TxLon = -111.888521012526
ensign_f__mhz = 3475.0
ensign_h_tx__meter = 19.9
ensign_h_rx__meter = 2.82
ensign_Elev_t_tx__meter = 1516.2

Look at the first five rows of the data.

In [ ]:
slc_ensign_df.head(5)

### Column definitions 

| Column            | Definition                         |
| ---------------   |------------------------------------|
| `ID`              | Identifier                         |
| `RxLat`           | Rx Latitude                        |
| `RxLon`           | Rx Longitude                       |
| `d__km`           | Distance TX to RX                  |
| `L_btl__db`       | Basic Transmission Loss            |
| `Elev_t_rx__meter`| Elevation at the RX Location       |
| `timeOffset__sec` | Time between measurements          |


### Calculate and add a new column for the RX elevation angle to the DataFrame

In [ ]:
## RX elevation angle = arcsin( (tx_altitude - rx_altitude) / Hypotenuse distance )
##  returns Rx elevation angle in degrees
def rx_elev_angle(alt_rx, alt_tx, d, antenna_rx_m, antenna_tx_m):
    soh = np.abs((alt_tx+antenna_tx_m) - (alt_rx+antenna_rx_m)) / (d*1000)
    angle_deg = np.arcsin(soh)*360/(np.pi*2) 
    return angle_deg

## calculate the rx elevation angle and add a new column to the dataframe
slc_ensign_df["rx_angle__deg"] = slc_ensign_df.apply(lambda row: rx_elev_angle(row.Elev_t_rx__meter, ensign_Elev_t_tx__meter, row.d__km, ensign_h_rx__meter, ensign_h_tx__meter), axis=1)

In [ ]:
slc_ensign_df.head(5)

### Summary Statistics for RX Elevation Angle

In [ ]:
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("Ensign - RX elevation angle (deg)\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(slc_ensign_df["rx_angle__deg"]),
                                                                                     np.max(slc_ensign_df["rx_angle__deg"]),
                                                                                     np.mean(slc_ensign_df["rx_angle__deg"]),
                                                                                     np.std(slc_ensign_df["rx_angle__deg"])))

### Calculate 3D Clutter Distance and add a new column
Remember the 3D Clutter Distance calculation relies on the mean (19.3 m) and standard deviation (12.0 m) that we measured of the clutter heights in Downtown SLC.

In [ ]:
h_c = slc_clutter_height_mean + 2*slc_clutter_height_std

def clutter_distance(rep_clutter_height, rx_angle):
    return rep_clutter_height/np.sin(rx_angle*(2*np.pi)/360) ## 2*pi/360 converts from degrees to radians

## add the new clutter distance columns
slc_ensign_df["clutter_d__meter"] = slc_ensign_df.apply(lambda row: clutter_distance(h_c, row.rx_angle__deg), axis=1)

In [ ]:
slc_ensign_df.head(5)

### Summary Statistics for 3D Clutter Distance

In [ ]:
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("Ensign - 3D Clutter Dist (meters)\t{:.1f}\t{:.1f}\t{:.1f}\t{:.1f}".format(np.min(slc_ensign_df["clutter_d__meter"]),
                                                                                     np.max(slc_ensign_df["clutter_d__meter"]),
                                                                                     np.mean(slc_ensign_df["clutter_d__meter"]),
                                                                                     np.std(slc_ensign_df["clutter_d__meter"])))

### Calculate the Clutter Loss and add a new column
The "clutter loss" column will be called `L_excess__db` (same as lesson 3). It is defined as "measured loss in excess of free space." Keep in mind that this dataset (and Browning, and City Creek) contains some locations where terrain obstructions exist. There aren't many, but it is present in the data. 

In [ ]:
## calculate Free Space Path Loss
def fspl(distance_km, frequency_mhz):
    fspl_lin = np.square((4 * np.pi * distance_km*1e3 * frequency_mhz*1e6) / 300e6)
    fspl_dB = 10 * np.log10(fspl_lin)
    return fspl_dB
## add Free Space Path Loss to the DataFrame    
slc_ensign_df["L_fs__db"] = slc_ensign_df.apply(lambda row: fspl(row.d__km, ensign_f__mhz), axis=1)

## add clutter loss = (basic transmission loss - free space path loss) to the DataFrame
slc_ensign_df["L_excess__db"] = slc_ensign_df.apply(lambda row: row.L_btl__db - row.L_fs__db, axis=1)

In [ ]:
slc_ensign_df.head(5)

### Plot the Path Distance vs Clutter Loss

In [ ]:
plt.scatter(slc_ensign_df["d__km"], slc_ensign_df["L_excess__db"], label='SLC Ensign', s=12)

plt.xlabel('Distance (km)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Path Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

### Plot the 3D Clutter Distance vs Clutter Loss

In [ ]:
plt.scatter(slc_ensign_df["clutter_d__meter"], slc_ensign_df["L_excess__db"], label='SLC Ensign', s=12)

plt.xlabel('3D Clutter Distance (m)')
plt.ylabel('Clutter Loss (dB)')
plt.title('3D Clutter Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

### Compare with our model (from lesson 3)

In [ ]:
## define the Lesson 3 model
y_int = -9.8
slope = 13.7
print("a = {:.2f} \nb = {:.2f}".format(slope, y_int))
print("L_cm = {:.2f} * log10(r_c) + {:.2f}".format(slope, y_int))

In [ ]:
## Plot
full_x = np.linspace(5,1750,2000)
full_y = slope*np.log10(full_x) + y_int
plt.plot(full_x, full_y, label='Model', c='b')

plt.scatter(slc_ensign_df["clutter_d__meter"], slc_ensign_df["L_excess__db"], label='SLC Ensign', s=8)

plt.xlabel('3D Clutter Distance (m)', fontsize=15)
plt.ylabel('Clutter Loss (dB)', fontsize=15)
plt.title('3D Clutter Distance Model vs SLC Data', fontsize=16)
plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

### Uh oh!

Let's look at the City Creek and Browning before we freak out.

# Dataset #2: City Creek
We'll do the same as Ensign. 
1. Bring in the data.
2. Calculate Rx elevation angle, 3D clutter distance, and clutter loss.
3. Plot the data vs our model.

In [ ]:
slc_citycreek_name = "./data/SaltLakeCity_Urban_CityCreek_3475_20230710.csv"
slc_citycreek_df = pd.read_csv(slc_citycreek_name)

citycreek_TxLat = 40.80719231
citycreek_TxLon = -111.8807371
citycreek_f__mhz = 3475.0
citycreek_h_tx__meter = 10.52
citycreek_h_rx__meter = 2.82
citycreek_Elev_t_tx__meter = 1875.6

In [ ]:
## calculate the rx elevation angle and add a new column to the dataframe
slc_citycreek_df["rx_angle__deg"] = slc_citycreek_df.apply(lambda row: rx_elev_angle(row.Elev_t_rx__meter, citycreek_Elev_t_tx__meter, row.d__km, citycreek_h_rx__meter, citycreek_h_tx__meter), axis=1)

## Summary of RX elevation angle
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("City Creek - RX elevation angle (deg)\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(slc_citycreek_df["rx_angle__deg"]),
                                                                                     np.max(slc_citycreek_df["rx_angle__deg"]),
                                                                                     np.mean(slc_citycreek_df["rx_angle__deg"]),
                                                                                     np.std(slc_citycreek_df["rx_angle__deg"])))

In [ ]:
## add the new clutter distance columns
slc_citycreek_df["clutter_d__meter"] = slc_citycreek_df.apply(lambda row: clutter_distance(h_c, row.rx_angle__deg), axis=1)

## Summary of 3D clutter distance
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("City Creek - 3D Clutter Dist (meter)\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(slc_citycreek_df["clutter_d__meter"]),
                                                                                     np.max(slc_citycreek_df["clutter_d__meter"]),
                                                                                     np.mean(slc_citycreek_df["clutter_d__meter"]),
                                                                                     np.std(slc_citycreek_df["clutter_d__meter"])))

In [ ]:
## add free space path loss
slc_citycreek_df["L_fs__db"] = slc_citycreek_df.apply(lambda row: fspl(row.d__km, citycreek_f__mhz), axis=1)
# add clutter loss (aka L_excess__db)
slc_citycreek_df["L_excess__db"] = slc_citycreek_df.apply(lambda row: row.L_btl__db - row.L_fs__db, axis=1)

In [ ]:
## plot clutter loss vs path distance
plt.scatter(slc_citycreek_df["d__km"], slc_citycreek_df["L_excess__db"], label='SLC City Creek', s=12, c='tab:orange')

plt.xlabel('Distance (km)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Path Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

In [ ]:
## plot 3D clutter distance vs clutter loss
plt.scatter(slc_citycreek_df["clutter_d__meter"], slc_citycreek_df["L_excess__db"], label='SLC City Creek', s=12, c='tab:orange')

plt.xlabel('Clutter distance (m)')
plt.ylabel('Clutter Loss (dB)')
plt.title('3D Clutter Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

In [ ]:
full_x = np.linspace(5,1000,2000)
full_y = slope*np.log10(full_x) + y_int
plt.plot(full_x, full_y, label='Model', c='b')

plt.scatter(slc_ensign_df["clutter_d__meter"], slc_ensign_df["L_excess__db"], label='SLC Ensign', s=5, c='tab:blue')
plt.scatter(slc_citycreek_df["clutter_d__meter"], slc_citycreek_df["L_excess__db"], label='SLC City Creek', s=5, c='tab:orange')


plt.xlabel('3D Clutter Distance (m)', fontsize=15)
plt.ylabel('Clutter Loss (dB)', fontsize=15)
plt.title('3D Clutter Distance Model vs SLC Data', fontsize=16)
plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

# Dataset #3: Browning

In [ ]:
slc_browning_name = "./data/SaltLakeCity_Urban_Browning_3475_20230710.csv"
slc_browning_df_full = pd.read_csv(slc_browning_name)
slc_browning_df = slc_browning_df_full.where(slc_browning_df_full["ID"] < 63301)

browning_TxLat = 40.766314982949
browning_TxLon = -111.847887797405
browning_f__mhz = 3475.0
browning_h_tx__meter = 35.2
browning_h_rx__meter = 2.82
browning_Elev_t_tx__meter = 1433.7

In [ ]:
## calculate the rx elevation angle and add a new column to the dataframe
slc_browning_df["rx_angle__deg"] = slc_browning_df.apply(lambda row: rx_elev_angle(row.Elev_t_rx__meter, browning_Elev_t_tx__meter, row.d__km, browning_h_rx__meter, browning_h_tx__meter), axis=1)

## Summary of RX elevation angle
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("Browning - RX elevation angle (deg)\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(slc_browning_df["rx_angle__deg"]),
                                                                                     np.max(slc_browning_df["rx_angle__deg"]),
                                                                                     np.mean(slc_browning_df["rx_angle__deg"]),
                                                                                     np.std(slc_browning_df["rx_angle__deg"])))

In [ ]:
## add the new clutter distance columns
slc_browning_df["clutter_d__meter"] = slc_browning_df.apply(lambda row: clutter_distance(h_c, row.rx_angle__deg), axis=1)

## Summary of 3D Clutter Distance
print("\t\t\t\t\tMin\tMax\tMean\tSt Dev")

print("Browning - 3D Clutter Dist (meter)\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(np.min(slc_browning_df["clutter_d__meter"]),
                                                                                     np.max(slc_browning_df["clutter_d__meter"]),
                                                                                     np.mean(slc_browning_df["clutter_d__meter"]),
                                                                                     np.std(slc_browning_df["clutter_d__meter"])))

In [ ]:
## add free space path loss
slc_browning_df["L_fs__db"] = slc_browning_df.apply(lambda row: fspl(row.d__km, browning_f__mhz), axis=1)
## add clutter loss (aka L_excess__db)
slc_browning_df["L_excess__db"] = slc_browning_df.apply(lambda row: row.L_btl__db - row.L_fs__db, axis=1)

In [ ]:
## plot clutter loss vs path distance 
plt.scatter(slc_browning_df["d__km"], slc_browning_df["L_excess__db"], label='SLC Browning', s=12, c='tab:green')

plt.xlabel('Distance (km)')
plt.ylabel('Clutter Loss (dB)')
plt.title('Path Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

In [ ]:
## plot 3D clutter distance vs Clutter loss
plt.scatter(slc_browning_df["clutter_d__meter"], slc_browning_df["L_excess__db"], label='SLC Browning', s=12, c='tab:green')

plt.xlabel('Clutter distance (m)')
plt.ylabel('Clutter Loss (dB)')
plt.title('3D Clutter Distance vs Clutter Loss')

plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

In [ ]:
## plot all three dataset vs our model
full_x = np.linspace(5,2300,2500)
full_y = slope*np.log10(full_x) + y_int
plt.plot(full_x, full_y, label='Model', c='b')

# plt.scatter(bd_df["clutter_d__meter"], bd_df["L_excess__db"], label='Boulder Downtown', s=5)
plt.scatter(slc_ensign_df["clutter_d__meter"], slc_ensign_df["L_excess__db"], label='SLC Ensign', s=5, c='tab:blue')
plt.scatter(slc_citycreek_df["clutter_d__meter"], slc_citycreek_df["L_excess__db"], label='SLC City Creek', s=5, c='tab:orange')
plt.scatter(slc_browning_df["clutter_d__meter"], slc_browning_df["L_excess__db"], label='SLC Browning', s=5, c='tab:green')


plt.xlabel('3D Clutter Distance (m)', fontsize=15)
plt.ylabel('Clutter Loss (dB)', fontsize=15)
plt.title('3D Clutter Distance Model vs SLC Data', fontsize=16)
plt.legend(fontsize=14)
plt.gca().yaxis.grid(True)
plt.show()

That's all three datasets. As you can see there are new challenges to applying this model to a new dataset with a different type of clutter. 

### Final Thoughts

We've done a lot. You've worked with lidar to get clutter metrics. You've thought about clutter as a slab that sits on top of the terrain. You've built a model that is based on RX elevation angle and local clutter heights. You've dug through five sets of measurement data (Martin Acres Low, Martin Acres High, Ensign, City Creek, and Browning). And you've compared your model to the P.2108 clutter model.

That leaves us with a __challenge__. As stated before, "In what ways does the model fail when tested against the Salt Lake City datasets? What can be done to improve it?"

Feel free to explore in any direction you choose. If you would like some ideas about what to try next we have some thoughts. Feel free to reach out to code@ntia.gov with breakthroughs or questions. 

End of Lesson 5. 